In [1]:
import pyspark.sql.functions as func
from time import time

Waiting for a Spark session to start...

Waiting for a Spark session to start...

In [2]:
spark

In [3]:
spark.sql("show databases").show()

+--------------------+
|        databaseName|
+--------------------+
|             backend|
|            backend1|
|             default|
|       demodb00_hcat|
|       demodb01_hcat|
|       demodb03_hcat|
|         demodb_hcat|
|                  go|
|              movies|
|            navntest|
|           navntest1|
|           perf_tpch|
|            sameeeee|
|              sample|
|         sample_navn|
|sample_navn_witho...|
|                test|
|              test01|
|               test2|
|         test_navn01|
+--------------------+
only showing top 20 rows



In [4]:
# crosstab pivoted specifications
# dimension->1 (from the same table) "o_orderstatus"
# measure-> 1 count from orders
# pivot -> same table "o_orderpriority"
# dimension from orders(TPCH) as bdaordersperf 
t1 = time()


print('**********Printing Pivot Column Discrete values***************')
sqlString = "select o_orderpriority from tpch10.orders";
ds = spark.sql(sqlString)

distinctds = ds.distinct().sort("o_orderpriority")
distinctds=distinctds.limit(1000)
for value in distinctds.collect():
   print value[0]

dforders = spark.table("tpch10.orders")
for column in dforders.columns:
   dforders = dforders.withColumnRenamed(column,"tpch10_orders_" + column)
rows = dforders.groupBy("tpch10_orders_o_orderstatus").pivot("tpch10_orders_o_orderpriority").agg(func.count('tpch10_orders_ROW_NUM')).sort("tpch10_orders_o_orderstatus")

rows1 = dforders.groupBy("tpch10_orders_o_orderstatus").agg(func.count('tpch10_orders_ROW_NUM')).sort("tpch10_orders_o_orderstatus") 
#rows1.show();

rows = rows.join(rows1,rows.tpch10_orders_o_orderstatus == rows1.tpch10_orders_o_orderstatus,"inner")

rows = rows.limit(1000);
print('****** Printing Actual Dataset ******************************')
for row in rows.collect():
    print row[0], row[1],  row[2], row[3], row[4], row[5], row[6], row[7]

print('*************Total count ************************************')
print rows.count()

totals = dforders.groupBy().pivot("tpch10_orders_o_orderpriority").agg(func.count('tpch10_orders_ROW_NUM'))
firstRowTotals = totals.first()
print('****** Printing Total  Dataset ******************************')
print firstRowTotals[0],firstRowTotals[1],firstRowTotals[2],firstRowTotals[3],firstRowTotals[4]

totals1 = dforders.groupBy().agg(func.count('tpch10_orders_ROW_NUM'))
firstRow1 = totals1.first()
print firstRow1[0]

print('****** Printing Time taken ******************************')
print time() - t1

**********Printing Pivot Column Discrete values***************
1-URGENT
2-HIGH
3-MEDIUM
4-NOT SPECIFIED
5-LOW
****** Printing Actual Dataset ******************************
F 1462766 1461800 1462276 1461756 1460586 F 7309184
O 1464013 1461240 1459703 1461850 1460658 O 7307464
P 76314 77021 76961 76654 76402 P 383352
*************Total count ************************************
3
****** Printing Total  Dataset ******************************
3003093 3000061 2998940 3000260 2997646
15000000
****** Printing Time taken ******************************
29.819247961


In [5]:
# crosstab pivoted specifications
# dimension->1 (from the same table) "o_orderstatus"
# measure-> 1 mean('o_totalprice') from orders
# pivot -> same table "o_orderpriority"
# dimension from orders(TPCH) as orders 

t1 = time()

print('**********Printing Pivot Column Discrete values***************')
sqlString = "select o_orderpriority from tpch10.orders";
ds = spark.sql(sqlString)
distinctds = ds.distinct().sort("o_orderpriority")
distinctds=distinctds.limit(1000)
for value in distinctds.collect():
   print value[0]


dforders = spark.table("tpch10.orders")
for column in dforders.columns:
   dforders = dforders.withColumnRenamed(column,"tpch10_orders_" + column)
rows = dforders.groupBy("tpch10_orders_o_orderstatus")\
             .pivot("tpch10_orders_o_orderpriority").agg(func.mean('tpch10_orders_o_totalprice'))\
             .sort("tpch10_orders_o_orderstatus")

#rows.show();
rows1 = dforders.groupBy("tpch10_orders_o_orderstatus")\
             .agg(func.mean('tpch10_orders_o_totalprice'))\
             .sort("tpch10_orders_o_orderstatus") 
#rows1.show();
rows = rows.join(rows1,rows.tpch10_orders_o_orderstatus == rows1.tpch10_orders_o_orderstatus,"inner")
#rows.show();
rows = rows.limit(1000);
print('****** Printing Actual Dataset ******************************')
for row in rows.collect():
    print row[0], row[1],  row[2], row[3], row[4], row[5], row[6], row[7]
    
print('*************Total count ************************************')
print rows.count()

totals = dforders.groupBy().pivot("tpch10_orders_o_orderpriority").agg(func.mean('tpch10_orders_o_totalprice'))

print('****** Printing Total  Dataset ******************************')
firstRowTotals = totals.first()
print firstRowTotals[0],firstRowTotals[1],firstRowTotals[2],firstRowTotals[3],firstRowTotals[4]

totals1 = dforders.groupBy().agg(func.mean('tpch10_orders_o_totalprice'))
firstRow1 = totals1.first()
print firstRow1[0]

print('****** Printing Time taken ******************************')
print time() - t1

**********Printing Pivot Column Discrete values***************
1-URGENT
2-HIGH
3-MEDIUM
4-NOT SPECIFIED
5-LOW
****** Printing Actual Dataset ******************************
F 150217.953676 150360.2419 150107.554346 150142.456041 150265.569962 F 150218.740521
O 150218.541297 150173.386023 150191.375195 150240.910075 150093.931562 O 150183.652375
P 184776.741132 185048.370787 184401.316909 184914.398212 185078.166727 P 184843.545656
*************Total count ************************************
3
****** Printing Total  Dataset ******************************
151096.441152 151159.783341 151028.424989 151078.819384 151069.216228
151086.54605
****** Printing Time taken ******************************
20.6066679955
